In [1]:
import argparse, os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:
parser = argparse.ArgumentParser(description="IMDN")
parser.add_argument("--block_size", type=int, default=64,
                    help="Block Size")
parser.add_argument("--crop_depth", type=int, default=30,
                    help="crop across z-axis")
parser.add_argument("--dir", type=str,
                    help="dataset_directory")
parser.add_argument("--batch_size", type=int,
                    help="dataset_directory")

_StoreAction(option_strings=['--batch_size'], dest='batch_size', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, required=False, help='dataset_directory', metavar=None)

In [3]:
args = list(parser.parse_known_args())[0]

In [4]:
args.dir = "/storage/users/arihant"

In [5]:
args.batch_size = 4

In [6]:
args

Namespace(block_size=64, crop_depth=30, dir='/storage/users/arihant', batch_size=4)

In [7]:
import data.HCP_dataset

In [8]:
import utils
ids = utils.get_ids()

number of common Subjects  171


In [9]:
training_dataset = data.HCP_dataset.hcp_data(args,ids)

In [10]:
training_dataset.tot

171

In [11]:
import numpy as np
def worker_init_fn(worker_id):
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset
    tot_len = len(dataset.ids)
    per_worker = int(tot_len / float(worker_info.num_workers))
    worker_id = worker_info.id
    dataset.ids = dataset.ids[worker_id * per_worker:max(((worker_id+1) * per_worker),tot_len )]
    dataset.curr_indx_blk = 0

    
def custom_collate(data):
    data = np.stack([data]).squeeze()
    data_gt = data[...,:8]
    data_pred = data[...,8:]
    return data_gt,data_pred

In [12]:
training_dataloader = torch.utils.data.DataLoader(training_dataset, num_workers=0,batch_size = 60,worker_init_fn=worker_init_fn,collate_fn=custom_collate)

In [13]:
for i in range(3):
    data = iter(training_dataloader)
    for j in data:
        print(j[0].shape,j[1].shape)

-1
100610 0


/home/arihant/DWI_SR/HCP_SR/data/utils_dataloader.py:147: RuntimeWarning: invalid value encountered in arccos
  ang_error = np.degrees(np.arccos(abs(dsm_rot @ dirs.T)))


(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
(4, 64, 64, 64, 9) (4, 64, 64, 64, 7)
0


KeyboardInterrupt: 